In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import csv
import tempfile
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
import numpy as np


import tensorflow as tf

FLAGS = None

In [2]:
data = np.load('data.npz')
train_X = data['train_X']
train_Y = data['train_y']
test_X = data['test_X']
    
x_train = train_X / 255
# x_train[x_train < 0.5] = 0
x_test = test_X / 255
# x_test[x_test < 0.5] = 0

# x_train = train_X
# x_test = test_X

In [3]:
def deepnn(x):
#   """deepnn builds the graph for a deep net for classifying digits.
#   Args:
#     x: an input tensor with the dimensions (N_examples, 784), where 784 is the
#     number of pixels in a standard MNIST image.
#   Returns:
#     A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
#     equal to the logits of classifying the digit into one of 10 classes (the
#     digits 0-9). keep_prob is a scalar placeholder for the probability of
#     dropout.
#   """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 64, 64, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([10, 10, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([16 * 16 * 64, 2048])
        b_fc1 = bias_variable([2048])
        
        h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([2048, 10])
        b_fc2 = bias_variable([10])
        
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    return y_conv, keep_prob

In [4]:
def conv2d(x, W):
#   """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
#   """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
#   """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
#   """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:

  # Import data
#   mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)



  # Create the model
x = tf.placeholder(tf.float32, [None, 4096])

  # Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

  # Build the graph for the deep net
y_conv, keep_prob = deepnn(x)

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv)
    cross_entropy = tf.reduce_mean(cross_entropy)

with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
        
        
accuracy = tf.reduce_mean(correct_prediction)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('Iteration begin')
for i in range(10000):
    X_sparse = coo_matrix(x_train)
    x_batch, X_sparse, y = shuffle(x_train, X_sparse, train_Y,n_samples=50)
    b = np.zeros((50, 10))
    for t in range(0,50):
        b[t, int(y[t])] = 1
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:x_batch, y_: b, keep_prob: 1.0},session=sess)
        print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: x_batch, y_: b, keep_prob: 0.7},session=sess)

Iteration begin
step 0, training accuracy 0.06
step 100, training accuracy 0.74
step 200, training accuracy 0.84
step 300, training accuracy 0.84
step 400, training accuracy 0.94
step 500, training accuracy 0.9
step 600, training accuracy 0.82
step 700, training accuracy 0.94
step 800, training accuracy 0.88
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 0.86
step 1200, training accuracy 0.94
step 1300, training accuracy 0.94
step 1400, training accuracy 0.98
step 1500, training accuracy 0.92
step 1600, training accuracy 0.94
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.94
step 2000, training accuracy 1
step 2100, training accuracy 1
step 2200, training accuracy 0.98
step 2300, training accuracy 1
step 2400, training accuracy 0.96
step 2500, training accuracy 0.98
step 2600, training accuracy 0.98
step 2700, training accuracy 0.94
step 2800, training accuracy 1
step 2900, training acc

In [ ]:
predictions = tf.argmax(y_conv, 1)
test_pred = predictions.eval(feed_dict={x: x_test, keep_prob: 1.0}, session=sess)
print(test_pred)

In [ ]:
print(test_pred[0])


In [ ]:
with open('result_10000.csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)
    wr.writerow(("Id","Label"))
    for index in range(len(test_pred)):
        wr.writerow(((index+1),test_pred[0]))